## Введение в обработку естественного языка

Домашнее задание №10

Урок 10. Машинный перевод. Модель seq2seq и механизм внимания

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

Разобраться с моделью перевода (без механизма внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)

В данном практическом задании напишу и обучу модель перевода с русского на английский
на библиотеке pytorch. Добавлю в неё механизм внимания, так как это сложнее и интереснее,
а также рекуррентную сеть энкодера сделаю двунаправленной.

Библиотеки

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

Датасет  

In [2]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2023-05-17 15:25:41--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15460248 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

     0K .......... .......... .......... .......... ..........  0%  136K 1m50s
    50K .......... .......... .......... .......... ..........  0%  272K 83s
   100K .......... .......... .......... .......... ..........  0%  516M 55s
   150K .......... .......... .......... .......... ..........  1%  921M 41s
   200K .......... .......... .......... .......... ..........  1%  272K 44s
   250K .......... .......... .......... .......... ..........  1%  201M 36s
   300K .......... .......... .......... .......... ..........  2%  132M 31s
   350K .......... .......... .......... .......... ..........  2%  273K 34s
   400K .......... .......... .......... ....

In [3]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [7]:
!ls ./rus-eng/ -lah

total 74M
drwxr-xr-x 1 jupyter jupyter   34 May 17 15:26 .
drwxr-xr-x 1 jupyter jupyter  118 May 17 15:26 ..
-rw-r--r-- 1 jupyter jupyter 1.5K Apr  2 03:16 _about.txt
-rw-r--r-- 1 jupyter jupyter  74M Apr  2 03:16 rus.txt


In [8]:
# Download the file
path_to_file = "./rus-eng/rus.txt"

In [65]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [66]:
preprocess_sentence("I can't fly like a bird.")

"<start> i can't fly like a bird . <end>"

In [67]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [68]:
en, ru = create_dataset(path_to_file, None)
print(en[42])
print(ru[42])

<start> wait ! <end>
<start> стой ! <end>


In [69]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [70]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)


In [71]:
len(en), len(ru)

(467119, 467119)

In [72]:
# Try experimenting with the size of that dataset
num_examples = 300000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [73]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

240000 240000 60000 60000


In [74]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [75]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
11 ----> ты
2850 ----> скучаешь
43 ----> по
558 ----> своим
1536 ----> детям
6 ----> ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
12 ----> do
6 ----> you
322 ----> miss
25 ----> your
446 ----> kids
7 ----> ?
2 ----> <end>


### Create a tf.data dataset

In [88]:
#!g1.1
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 200
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [89]:
#!g1.1
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 18]), TensorShape([64, 13]))

In [90]:
#!g1.1
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [91]:
#!g1.1
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

2023-05-17 15:42:13.940640: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101


Encoder Hidden state shape: (batch size, units) (64, 1024)


In [92]:
#!g1.1
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [93]:
#!g1.1
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)



In [94]:
#!g1.1
decoder_sample_x.shape

TensorShape([64, 12598])

In [95]:
#!g1.1
decoder_sample_h.shape

TensorShape([64, 1024])

## Define the optimizer and the loss function

In [96]:
#!g1.1
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) # маска будет зануляиь нулевые токены паддинга , 
            #через которые мы не хотим прокидывать градиент, 
            #поэтому мы их сразу в лоссе зануляем чтобы они на лосс никак не реагировали, 
            #чтобы не вносили джобавочные коэффициенты в наш лосс
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [97]:
#!g1.1
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [98]:
#!g1.1
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [99]:
#!g1.1

%%time

EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)): 
        # inp это текст на русском а targ это текст на англ - то что мы ходим получить
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

2023-05-17 15:42:54.444284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1 Batch 0 Loss 5.0723
Epoch 1 Batch 100 Loss 2.4849
Epoch 1 Batch 200 Loss 2.1388
Epoch 1 Batch 300 Loss 2.0370
Epoch 1 Batch 400 Loss 1.8355
Epoch 1 Batch 500 Loss 1.8472
Epoch 1 Batch 600 Loss 1.7284
Epoch 1 Batch 700 Loss 1.4948
Epoch 1 Batch 800 Loss 1.4560
Epoch 1 Batch 900 Loss 1.4138
Epoch 1 Batch 1000 Loss 1.3458
Epoch 1 Batch 1100 Loss 1.2454
Epoch 1 Batch 1200 Loss 1.2433
Epoch 1 Batch 1300 Loss 1.1861
Epoch 1 Batch 1400 Loss 1.3452
Epoch 1 Batch 1500 Loss 1.1816
Epoch 1 Batch 1600 Loss 1.0039
Epoch 1 Batch 1700 Loss 1.0431
Epoch 1 Batch 1800 Loss 0.9384
Epoch 1 Batch 1900 Loss 1.0764
Epoch 1 Batch 2000 Loss 1.0756
Epoch 1 Batch 2100 Loss 0.9501
Epoch 1 Batch 2200 Loss 1.0049
Epoch 1 Batch 2300 Loss 0.9930
Epoch 1 Batch 2400 Loss 0.7928
Epoch 1 Batch 2500 Loss 0.9512
Epoch 1 Batch 2600 Loss 0.7285
Epoch 1 Batch 2700 Loss 0.7570
Epoch 1 Batch 2800 Loss 0.8158
Epoch 1 Batch 2900 Loss 0.8160
Epoch 1 Batch 3000 Loss 0.7842
Epoch 1 Batch 3100 Loss 0.7642
Epoch 1 Batch 3200 L

In [100]:
#!g1.1
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [101]:
#!g1.1
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [102]:
#!g1.1
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Тестирование

In [107]:
#!g1.1
# Тестируем

test_texts = ['Здесь все классно.', 'Я не успеваю делать домашку.', 'Когда я буду много получать денег?',
              'Я учусь учить модели.', 'Береги природу, мать твою.',
              'Перевод можно сделать и лучше', 'Выберите новый ноутбук по Вашим возможностям.',
              ]

for text in test_texts:
    translate(text)

Input: <start> здесь все классно . <end>
Predicted translation: it's all here funny . <end> 
Input: <start> я не успеваю делать домашку . <end>
Predicted translation: i don't have time for my homework . <end> 
Input: <start> когда я буду много получать денег ? <end>
Predicted translation: when did i buy a lot of money ? <end> 
Input: <start> я учусь учить модели . <end>
Predicted translation: i'm making model history . <end> 
Input: <start> береги природу , мать твою . <end>
Predicted translation: please love your mother . <end> 
Input: <start> перевод можно сделать и лучше <end>
Predicted translation: the place can only be happy . <end> 
Input: <start> выберите новый ноутбук по вашим возможностям . <end>
Predicted translation: choose your new haircut . <end> 


Результаты ну такое себе, но даже немного забавно
